In [1]:
import pandas as pd
import math
import numpy as np

### Загрузка и обработка данных

In [2]:
df=pd.read_csv('Data.csv')

In [3]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%Y-%m-%d')
df['InvoiceDate'] = df['InvoiceDate'].dt.strftime('%Y-%m-%d')

In [4]:
df2=df[df.InvoiceDate.str.contains(r'2010-12-\d\d', regex= True, na=False)]

In [5]:
# df2=df[df['InvoiceDate']=='2010-12-23']

In [6]:
df2['Total']=df2['Quantity']*df2['Price']

C:\Users\12967\AppData\Local\Temp\ipykernel_10148\615049507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Total']=df2['Quantity']*df2['Price']


In [7]:
df_group=df2.groupby(['Customer ID', 'Country']).agg({'Total': ['sum']}). reset_index()
df_group.columns = df_group.columns.droplevel(1)

In [61]:
df_group=df_group[df_group['Country']!='United Kingdom']

### Полное совместное распределение

Интересующее событие - стоимость чека декабря 2010 года,
Событие наблюдение - страна 

In [64]:
def to_prob(df):
    df['Max_Price']=(df['Total']//100+1)*100
    df['Max_Price'] = df['Max_Price'].astype(int) 
    
    country=df['Country'].unique()
    max_price=df['Max_Price'].unique()
    
    df_country=pd.DataFrame(data=country, columns=['country'])
    df_country['key']=0
    
    df_max_price=pd.DataFrame(data=max_price, columns=['max_price'])
    df_max_price['key']=0
    
    df_prob=df_country.merge(df_max_price, on='key', how='outer')
    df_prob['probability']=0
    
    return df_prob

In [75]:
df_prob=to_prob(df_group)

In [76]:
len_df=len(df_group)

In [77]:
def count_row(country, max_price, df_search):
    df_temp=df_search[(df_search['Country']==country)&(df_search['Max_Price']==max_price)]
    return len(df_temp)

In [78]:
for i, row in df_prob.iterrows():
    country_temp=row['country']
    max_price_temp=row['max_price']
    res=count_row(country_temp, max_price_temp, df_group)
    df_prob.loc[i,'probability']= res/len_df

In [79]:
df_prob=df_prob[['country', 'max_price', 'probability']].sort_values('probability', ascending=False)

,country,max_price,probability
239,France,200,0.064103
169,Germany,0,0.051282
163,Germany,300,0.038462
170,Germany,200,0.038462
234,France,1100,0.038462
...,...,...,...
201,Denmark,500,0.000000
200,Denmark,-200,0.000000
199,Denmark,1200,0.000000
198,Denmark,1400,0.000000


### Частное распределение

#### P(country)

In [80]:
df_P_c=df_prob.groupby(['country']).agg({'probability': ['sum']}). reset_index()
df_P_c.columns = df_P_c.columns.droplevel(1)
df_P_c

,country,probability
0,Australia,0.038462
1,Austria,0.025641
2,Belgium,0.051282
3,Channel Islands,0.012821
4,Cyprus,0.012821
5,Denmark,0.012821
6,EIRE,0.038462
7,Finland,0.012821
8,France,0.192308
9,Germany,0.269231


#### P(max_price)

In [19]:
df_P_p=df_prob.groupby(['max_price']).agg({'probability': ['sum']}). reset_index()
df_P_p.columns = df_P_p.columns.droplevel(1)
df_P_p

,max_price,probability
0,-200,0.012821
1,0,0.089744
2,100,0.025641
3,200,0.179487
4,300,0.128205
5,400,0.076923
6,500,0.051282
7,700,0.051282
8,800,0.051282
9,900,0.051282


### Условное распределение

#### P(max_price<500 | country = Germany)

In [20]:
P_less500_and_Ger=df_prob[(df_prob['max_price']>500)&(df_prob['country']=='Germany')]['probability'].sum()

In [21]:
P_Ger=df_prob[df_prob['country']=='Germany']['probability'].sum()

In [22]:
P_less500_Ger=P_less500_and_Ger/P_Ger
P_less500_Ger

0.42857142857142855

### Прогнозирование

In [42]:
df2

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
496394,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
496395,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
496396,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
496397,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
496398,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
538370,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23,1.25,NaN,United Kingdom,1.25
538371,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23,1.45,NaN,United Kingdom,5.80
538372,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23,3.75,NaN,United Kingdom,3.75
538373,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23,1.25,NaN,United Kingdom,1.25


In [43]:
data=df_group[['Country', 'Total']]
data2=pd.get_dummies(data, dtype=float)

In [45]:
data=df2[['StockCode', 'Price', 'Country', 'Quantity']]
data2=pd.get_dummies(data, dtype=float)

In [46]:
data2

,Price,Quantity,StockCode_10002,StockCode_10120,StockCode_10123C,StockCode_10124A,StockCode_10124G,StockCode_10125,StockCode_10133,StockCode_10135,...,Country_Japan,Country_Lithuania,Country_Netherlands,Country_Norway,Country_Poland,Country_Portugal,Country_Spain,Country_Sweden,Country_Switzerland,Country_United Kingdom
496394,2.55,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
496395,3.39,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
496396,2.75,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
496397,3.39,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
496398,3.39,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538370,1.25,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
538371,1.45,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
538372,3.75,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
538373,1.25,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [47]:
X=data2.drop(['Quantity'], axis=1)
y=data2['Quantity']

In [48]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 
 

In [49]:
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier() 
rf.fit(X_train, y_train) 

RandomForestClassifier()

In [50]:
pred=rf.predict(X_test)

In [52]:
len(pred)

12595

In [53]:
ind=X_test.index
ind

Int64Index([513388, 502074, 523574, 520033, 538309, 519405, 521724, 513129,
            535389, 497089,
            ...
            529864, 511449, 512274, 509388, 533346, 497036, 518224, 498539,
            507000, 529276],
           dtype='int64', length=12595)

In [54]:
df2

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total
496394,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
496395,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
496396,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
496397,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
496398,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
538370,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23,1.25,NaN,United Kingdom,1.25
538371,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23,1.45,NaN,United Kingdom,5.80
538372,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23,3.75,NaN,United Kingdom,3.75
538373,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23,1.25,NaN,United Kingdom,1.25


In [56]:
df_new=df2.loc[ind][['Country','StockCode', 'Price']]
df_new['Quantity']=pred
df_new['Total']=df_new['Price']*df_new['Quantity']

,Country,StockCode,Price,Quantity,Total
513388,United Kingdom,21976,0.55,5,2.75
502074,United Kingdom,22118,11.87,2,23.74
523574,United Kingdom,22375,8.47,1,8.47
520033,United Kingdom,22212,2.10,4,8.40
538309,United Kingdom,82613A,1.25,20,25.00
...,...,...,...,...,...
497036,United Kingdom,22951,0.55,24,13.20
518224,United Kingdom,22778,8.47,1,8.47
498539,United Kingdom,22622,9.95,1,9.95
507000,United Kingdom,84712B,7.95,1,7.95


In [59]:
df_group_new=df_new.groupby(['Country']).agg({'Total': ['sum']}). reset_index()
df_group_new.columns = df_group_new.columns.droplevel(1)
df_group_new=df_group_new[df_group_new['Country']!='United Kingdom']

In [110]:
df_prob_new=to_prob(df_group_new)
len_df_new=len(df_group_new)

In [111]:
for i, row in df_prob_new.iterrows():
    country_temp=row['country']
    max_price_temp=row['max_price']
    res=count_row(country_temp, max_price_temp, df_group_new)
    df_prob_new.loc[i,'probability']= res/len_df

In [112]:
df_prob_new

,country,key,max_price,probability
0,Australia,0,300,0.012821
1,Australia,0,100,0.000000
2,Australia,0,600,0.000000
3,Australia,0,400,0.000000
4,Australia,0,2600,0.000000
...,...,...,...,...
268,Switzerland,0,800,0.000000
269,Switzerland,0,2800,0.000000
270,Switzerland,0,1100,0.000000
271,Switzerland,0,700,0.000000


In [114]:
df_prob_new=df_prob_new[['country', 'max_price', 'probability']].sort_values('probability', ascending=False)

In [115]:
P_less500_and_Ger_new=df_prob_new[(df_prob_new['max_price']<500)&(df_prob_new['country']=='Germany')]['probability'].sum()
P_less500_and_Ger_new

0.0

In [116]:
P_Ger_new=df_prob_new[df_prob_new['country']=='Germany']['probability'].sum()
P_Ger_new

0.01282051282051282

In [117]:
P_less500_Ger_new=P_less500_and_Ger_new/P_Ger_new
P_less500_Ger_new

0.0

### Оценка

In [118]:
from sklearn.metrics import classification_report 
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

       -9360       0.00      0.00      0.00         1
        -300       0.00      0.00      0.00         1
        -240       0.00      0.00      0.00         0
        -223       0.00      0.00      0.00         1
        -192       0.00      0.00      0.00         1
        -144       0.00      0.00      0.00         1
         -96       0.00      0.00      0.00         0
         -80       0.00      0.00      0.00         2
         -50       0.00      0.00      0.00         1
         -48       0.00      0.00      0.00         2
         -40       0.00      0.00      0.00         1
         -38       0.00      0.00      0.00         0
         -36       0.00      0.00      0.00         2
         -33       0.00      0.00      0.00         2
         -32       0.00      0.00      0.00         1
         -30       0.00      0.00      0.00         1
         -25       0.00      0.00      0.00         1
         -24       0.00    

C:\Users\12967\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\12967\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\12967\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\12967\anaconda3\lib\site-packages\sklearn\metrics\_cla